In [2]:
# STS
import pandas as pd
from csv import QUOTE_NONE
data_dir = "./data/glue_data/STS-B/"
def read_table(loc):
    return pd.read_csv(loc, sep='\t', index_col=0, quoting=QUOTE_NONE)
train = read_table(data_dir+"train.tsv")
dev = read_table(data_dir+"dev.tsv")
test = read_table(data_dir+"test.tsv")

In [33]:
train_input1 = train['sentence1'].tolist()
train_input2 = train['sentence2'].tolist()
train_input = list(zip(train_input1, train_input2))
train_output = train['score'].tolist()
dev_input = list(zip(dev['sentence1'].tolist(), dev['sentence2'].tolist()))
dev_output = dev['score'].tolist()

# Similarity based on co-existance.
but simple and naive

In [34]:
import nltk
nltk.download('punkt')
from nltk import word_tokenize
from nltk.tokenize import sent_tokenize
tkn = lambda x: word_tokenize(sent_tokenize(x)[0])

[nltk_data] Downloading package punkt to
[nltk_data]     /slfs1/users/xyl98/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [43]:
def similarity(s1, s2, tkn=word_tokenize):
    ws1 = [w.lower() for w in tkn(s1) if w != '.']
    ws2 = [w.lower() for w in tkn(s2) if w != '.']
    mark1 = 0.5 * len([w for w in ws1 if w in ws2]) / len(ws1)
    mark2 = 0.5 * len([w for w in ws2 if w in ws1]) / len(ws2)
    return mark1+mark2

In [44]:
def calc(s1, s2):
    return 5*similarity(s1, s2)

In [45]:
for i in range(0, 10):
    s1, s2 = train_input[i]
    print(s1)
    print(s2)
    print("Expect:\t", train_output[i])    
    print("Actual:\t", calc(s1, s2))
    print()

A plane is taking off.
An air plane is taking off.
Expect:	 5.0
Actual:	 3.666666666666667

A man is playing a large flute.
A man is playing a flute.
Expect:	 3.8
Actual:	 4.642857142857143

A man is spreading shreded cheese on a pizza.
A man is spreading shredded cheese on an uncooked pizza.
Expect:	 3.8
Actual:	 3.972222222222222

Three men are playing chess.
Two men are playing chess.
Expect:	 2.6
Actual:	 4.0

A man is playing the cello.
A man seated is playing the cello.
Expect:	 4.25
Actual:	 4.642857142857143

Some men are fighting.
Two men are fighting.
Expect:	 4.25
Actual:	 3.75

A man is smoking.
A man is skating.
Expect:	 0.5
Actual:	 3.75

The man is playing the piano.
The man is playing the guitar.
Expect:	 1.6
Actual:	 4.166666666666667

A man is playing on a guitar and singing.
A woman is playing an acoustic guitar and singing.
Expect:	 2.2
Actual:	 3.611111111111111

A person is throwing a cat on to the ceiling.
A person throws a cat on the ceiling.
Expect:	 5.0
Actual

## Evaluation

In [54]:
import numpy as np
import scipy
from scipy.stats import pearsonr
def evaluate(output, pred):
    return pearsonr(output, pred)[0]

In [55]:
train_pred = [calc(s1, s2) for s1, s2 in train_input]
evaluate(train_output, train_pred)

0.5609705563764735

In [56]:
dev_pred = [calc(s1, s2) for s1, s2 in dev_input]
evaluate(dev_output, dev_pred)

0.6208314538664363